# SQLAlchemy

SQLAlchemy is a very common used ORM (Object Relational Mapper), allowing us to access the data in an object-oriented way. (It actually allows to use different databases without changing our Python code).

Note: in this Notebook we will use the Flask built-in SQLAlchemy package:

`pip install flask-sqlalchemy`

In [2]:
from flask import Flask

## Step 0: Initialization

Import the `sqlalchemy` module:

In [3]:
from flask_sqlalchemy import SQLAlchemy

To start, we will use a light database that will be stored in the local machine. We set up the URI of where this file will be stored by setting:

In [4]:
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'database_URI'

`'database_URI'` is the actual URI where our database file can be found. We can specify it, or create a new one inserting `sqlite:///newdb.db` instead.
(`:///` means our current directory, and the command will create a new database file called `newdb.db`)

## Step 1: Creating database instances

Note: the instance should be created in a separate `models` file. 'Models' are database classes, which will be its own table in the database.

In [5]:
db = SQLAlchemy(app)

//anaconda3/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


This statement is used to control the SQLAlchemy integration onto Flask (note that variable `app` is a Flask application).

`app = Flask(__name__)`

### Creating a 'User' class (table in the database)

In [24]:
class User(db.Model):
    id = db.Column(db.Integer,
                   primary_key = True)
    username = db.Column(db.String(20),
                         unique = True,
                         nullable = False)
    image_file = db.Column(db.String(20),
                           nullable = False,
                           default = 'defaul.jpg')
    password = db.Column(db.String(60),
                         nullable = False)
    
    def __repr__(self):
        return "User('{self.username}', '{self.email}', '{self.image_file}')"

```python
class User(db.Model):```
- Defines a new class called 'User' (we decide the name), inside the database 'db'.

```python
id = db.Column(db.Integer,
               primary_key = True)```
- Sets the 'id' as the first Column of the database, which requires an integer as input, and is set up as primary key (parameter set up to uniquely identify all table records) of the database.

```python
username = db.Column(db.String(20),
                     unique = True,
                     nullable = False)```
- Sets up 'username' as the second column of the database, which will be a string of at most 20 characters. `unique = True` means all users will need a distinct username, and `nullable = False` means all users will require a non-null username.

```python
def __repr__(self):
        return '...'```
- Specifies the way an instance of the class will be printed if the `print()` function is called on it.

### Creating a 'Post' Class

In [25]:
from datetime import datetime

class Post(db.Model):
    id = db.Column(db.Integer,
                   primary_key = True)
    title = db.Column(db.String(100),
                      nullable = False)
    date_posted = db.Column(db.DateTime,
                            nullable = False,
                            default = datetime.utcnow())
    content = db.Column(db.Text,
                        nullable = False)
    
    def __repr__(self):
        return "Post('{self.title}', '{self.date_posted}')"

```python
class Post(db.Model):```
- Defines a new class called 'Post' inside the database 'db'.

```python
id = db.Column(db.Integer,
               primary_key = True)```
- Sets the post 'id' as the first Column of the database, and is set up as primary key.

```python
date_posted = db.Column(db.DateTime,
                        nullable = False,
                        default = datetime.utcnow())```
- Sets up 'date_posted' as a `DateTime` object, non-nullable. `default` sets the default value of the `date_posted` column, which will be the actual time of the post if no time is specified.

## Step2: Adding relationships between classes

### Updating the 'User' class

In [6]:
# <Same as before>
class User(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    username = db.Column(db.String(20),unique = True,nullable = False)
    image_file = db.Column(db.String(20), nullable = False, default = 'default.jpg')
    password = db.Column(db.String(60), nullable = False)
    # </Same as before>
    
    # <New Code>
    posts = db.relationship('Post',
                            backref = 'author',
                            lazy = True)
    # </New Code>
    
    def __repr__(self):
        return f"User('{self.username}', '{self.email}', '{self.image_file}')"

```python
posts = db.relationship('Post',
                        backref = 'author',
                        lazy = True)```
Adds a `relationship` with the class (or model) `'Post'` (which needs to be specified as a string). Note that this won't create a `Column` in the database, but will allow us to query the posts done by that user. What `relationships` do is create additional operations in the background that will allow to get all posts that users have created; check **Step3 - g)**.

- The `backref` is similar to adding a column to the `Post` model (without doing it). What `backref` will allow is, for any Post, we will be able to retrieve its `User` instance by calling the `.author` method on it; check **Step3 - i)**.

- Adding a `lazy` argument will enable lazy loading of the data.


In [8]:
# <Same as before>
from datetime import datetime

class Post(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    title = db.Column(db.String(100),nullable = False)
    date_posted = db.Column(db.DateTime,nullable = False,default = datetime.utcnow())
    content = db.Column(db.Text,nullable = False)
    # </Same as before>
    
    # <New Code>
    user_id = db.Column(db.Integer,
                        db.ForeignKey('user.id'),
                        nullable = False)
    
    
    def __repr__(self):
        return f"Post('{self.title}', '{self.date_posted}')"

```python
user_id = db.Column(db.Integer,
                    db.ForeignKey('user.id'),
                    nullable = False)```
This creates a new Column that requires to be an `Integer`, and that will come from the `id` argument of the `user` table.

Note that we user lower-case `'user.id'` as a string instead of referencing the model `User` as we did for the `backref`. This is due to the fact that we need to reference the table name (and tables have always lower-case names).

## Step 3: add items to database

For this part, we will assume the above code is saved in `model.py` folder.

**a) Make sure to activate python on the command line**:

`$ python`

**b) Import the database created:**

`from model import db`

**c) Create the database:**

`db.create_all()`

This creates an empty database instance in the URI specified above (if we specified `sqlite:///newdb.d`, it will create it in a new file `newdb.db`).

Note: use `db.clear.all()` to clear all database tables.

**d) Creating a 'User' and 'Post' instance:**

`from model import User, Post`

`user_1 = (username = 'Lennon', email = 'john@lemon.com', password = 'imagine')`

`user_2 = (username = 'McCartney', email = 'paul@macca.com', password = 'letitbe')`

**e) Adding users to the database:**

`db.session.add(user_1)`

`db.session.add(user_2)`

`db.session.commit()`

These three commands add the user instances to the database

**f) Quering users:**

`User.query.all()` Will return a list with all users. (note syntax is `Model.query.all()`)

Note: the `.query.all()` method will return a list of all `Users` added in the database.

`User.query.first()` will return the first user.

`User.query.filter_by(username = 'McCartney')` will return a list of users satisfying the condition, in this case, the second one. (Here we will never obtain more than 1 result, since our `username` is unique, we can use the `.first()` method to get the user out of the list).

`User.query.get(id)` will return the user associated to the `id` inserted (it should be an `integer`).

**g) Getting conditions from users (including reference)**

Setting a variable equal to a user allows us to query all arguments of the user:

`user1 = User.query.filter_by(username = 'McCartney').first()`

Now we can query the other arguments defined in Steps 1 and 2:

`user1.id`

- Will return the ID associated to it (etc.)

`user.posts`

- Will return a list with the posts done by the user, this can be done thanks to setting `posts = db.relationship(...)` in **Step2**.

**h) Creating posts**

`post1 = Post(title='Post 1', content='First Content', user_id = user.id)`
`post2 = Post(title='Post 2', content='Second Content', user_id = user.id)`

Note that we have used the previously defined variable `user` to enter the `user_id` argument.

`db.session.add(post1)`

`db.session.add(post2)`

`db.session.commit()`

**i) Using the backref to query the user**

`post = Post.query.first()` associates a variable to the first post (now we can query it)

`post.author`

- Will return the User that created the author thanks to the `backreference` created in **Step2**.

**j) Deleting rows from database**

`db.session.delete(user1)`

`db.session.commit()`

Deletes `user1` (previously defined) from the database.